# RSI Divergence Analysis

In [2]:
%load_ext memory_profiler

In [3]:
import pandas as pd 
import numpy as np
import time
%memit np.zeros(100000)
%timeit np.zeros(100000)

peak memory: 99.29 MiB, increment: 0.94 MiB
13.4 µs ± 236 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [54]:
n=14
df = pd.read_csv("data/input/USDT_BTC_86400_1425168000_72.csv")
print(f"Dataframe shape:{df.shape}")
df = df.drop(columns=['ts','period','currency_pair','volume','quoteVolume','quoteVolume','weightedAverage'])
df.head()

Dataframe shape:(2192, 11)


,date,high,low,open,close
0,1425168000,251.00,240.000121,240.000121,251.00000
1,1425254400,269.00,251.000000,252.000000,269.00000
2,1425340800,274.50,255.000000,268.000000,267.22000
3,1425427200,280.00,258.875000,258.875000,280.00000
4,1425513600,280.89,257.000000,257.000000,257.00012


In [29]:
def get_rsi(df,n):
    
    delta = df.close - df.close.shift(1) 
    delta = delta.dropna()
    
    closeup = delta.mask(delta < 0, 0)
    closedown = delta.mask(delta > 0,0).abs()

    closeup[closeup.index[n-1]]= closeup[:n].mean()
    closeup = closeup.drop(closeup.index[:n-1])
    avg_closeup = closeup.ewm(com=n-1,adjust=False).mean()
  
    closedown[closedown.index[n-1]]= closedown[:n].mean()
    closedown = closedown.drop(closedown.index[:n-1])
    avg_closedown = closedown.ewm(com=n-1,adjust=False).mean()


    rsi = avg_closeup / (avg_closedown + avg_closeup)
    
    return rsi

In [55]:
def rsi_divergence(df):
    ema_price = df.close.ewm(span=13,min_periods=13).mean()
    ema_rsi = df.rsi.ewm(span=13,min_periods=12).mean()
    df['ema_price'] = ema_price
    df['ema_rsi'] = ema_rsi
    df['momentum_ema_price'] = df.ema_price - df.ema_price.shift(1)
    df['momentum_ema_rsi'] = df.ema_rsi - df.ema_rsi.shift(1)
    df['momentum_rsi'] = df.rsi - df.rsi.shift(1)
    df['bull_divergence_1'] = np.where((df.momentum_ema_price < 0) & (df.momentum_ema_rsi > 0),1 ,0)
    df['bull_divergence_2'] = np.where((df.momentum_ema_price < 0) & (df.momentum_rsi > 0),1 ,0)
    
    

In [56]:
df['rsi'] = get_rsi(df,n)
rsi_divergence(df)
df
df.to_csv("rsi_output.csv")
